## here we will use the langchain for tools and agents let's see how to do it 
### here is the link :-https://python.langchain.com/v0.1/docs/integrations/tools/

In [2]:
## Arxiv --> research
## Tools creations

from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

wiki_api = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
## this wrapper we use for fetching the information from that perticular website 
wiki = WikipediaQueryRun(api_wrapper=wiki_api) ## this queryrun we use to fetch the information
## this WIKI--> is my tool 
# we use inbuild tools of wikipedia 


In [3]:
Arxiv_api = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=Arxiv_api)
## arxiv is the another inbuild tool we are using from the langchain 

In [4]:
tools = [wiki,arxiv] # here we will extend/dalegai all those tools which we want to make 


In [5]:
## let's build the custome tool 

from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os 
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
loader  = WebBaseLoader("https://python.langchain.com/docs/integrations/providers/")
docs = loader.load()
split = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
documents = split.split_documents(docs)
emb = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")




USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\AKM\Documents\GenrativeAI\Differen_applications\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
vec = Chroma.from_documents(documents,emb)
retriver = vec.as_retriever()


### to convert the retriver into the tool we will use this 

In [28]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool = create_retriever_tool(retriver,"langchin-search","Search anything regarding the langchian.")
## this is the retriver tool which we made/customized like this we can make different tools
retriver_tool.name

'langchin-search'

In [56]:
tools.append(retriver_tool)

In [57]:
tools ## here you can see the discription of the tools 

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\AKM\\Documents\\GenrativeAI\\Differen_applications\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='langchin-search', description='Search anything regarding the langchian.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000241A9D047C0>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma ob

## Now we will see how to run all this tools with the Agents and LLM

In [50]:
## llm,tools --> AgentExecutor
from langchain_groq import ChatGroq
os.environ["GROQ_API"] = os.getenv("GROQ_API")
groq_api = os .getenv("GROQ_API")

llm = ChatGroq(api_key=groq_api,model="llama-3.1-8b-instant")

## tere are some prompt tamplet which are all ready created we are just calling those templates 
## you can see those things from 
## https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/


In [51]:
## calling the prompt template 
from langchain import hub 
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Now we will combine those prompt with the LLM and let's build the agent and let them interact with the 
## tools 
<h1><b>To execute those tools with llm and how should llm interact with those tools for that we use
<i>Agents<i><b></h1>

In [58]:
## Agents 
## see the link of the types of agents
## https://python.langchain.com/v0.1/docs/modules/agents/agent_types/

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [59]:
## Agent Executor 

from langchain.agents import AgentExecutor

agent_exe = AgentExecutor(agent=agent,tools=tools,verbose=True) ## verbose = True karnai se we can see how the interaction happens
agent_exe

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [65]:
agent_exe.invoke({"input":"what is llm"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'LLM'}`




c:\Users\AKM\Documents\GenrativeAI\Differen_applications\venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\AKM\Documents\GenrativeAI\Differen_applications\venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was found
Invoking: `langchin-search` with `{'query': 'LLM'}`


SystemsiFixitiFlytekIMSDbInfinispan VSInfinityInfinoIntelIuguJaguarJavelin AI GatewayJina AIJohnsnowlabsJoplinKDB.AIKineticaKoboldAIKonkoKoNLPYKùzuLabel StudiolakeFSLanceDBLangChain Decorators ✨LanternLinkupLiteLLMLlamaIndexLlama.cppLlamaEdgellamafileLLMonitorLocalAILog10MariTalkMarqoMediaWikiDumpMeilisearchMemcachedMetalMicrosoftMilvusMindsDBMinimaxMistralAIMLflow AI Gateway for LLMsMLflowMLXModalModelScopeModern TreasuryMomentoMongoDBMongoDB AtlasMotherduckMotörheadMyScaleNAVERNeo4jNLPCloudNomicNotion DBNucliaNVIDIAObsidianOceanBaseOracle Cloud Infrastructure (OCI)OctoAIOllamaOntotext GraphDBOpenAIOpenLLMOpenSearchOpenWeatherMapOracleAI Vector SearchOutlineOutlinesPandasPebbloPerplexityPetalsPostgres EmbeddingPGVectorPineconePipelineAIPortkeyPredibasePrediction GuardPremAIPromptLayerPsychicPubMedPygmalionAIQdrantRAGatouillerank_bm25Ray ServeRebuffRedditRedisRemembrallReplicateRoamSema4 (fk

c:\Users\AKM\Documents\GenrativeAI\Differen_applications\venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\AKM\Documents\GenrativeAI\Differen_applications\venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was foundAn LLM (Large Language Model) is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate human-like language. LLMs are designed to process and understand natural language, allowing them to perform tasks such as language translation, text summarization, and even generating creative content like stories or poetry.

LLMs work by taking in a prompt or input, and then generating a response based on the patterns and relationships it has learned from the training data. They can be fine-tuned for specific tasks, such as language translation or text classification, and can be used in a variety of applications, including chatbots, virtual assistants, and content generation tools.

There are many different types of LLMs, each with its own strengths and weaknesses. Some popular LLMs include:

* BERT (Bidirectional Encoder Representations from Transformers): A widely used LLM developed by Google that is known f

{'input': 'what is llm',
 'output': 'An LLM (Large Language Model) is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate human-like language. LLMs are designed to process and understand natural language, allowing them to perform tasks such as language translation, text summarization, and even generating creative content like stories or poetry.\n\nLLMs work by taking in a prompt or input, and then generating a response based on the patterns and relationships it has learned from the training data. They can be fine-tuned for specific tasks, such as language translation or text classification, and can be used in a variety of applications, including chatbots, virtual assistants, and content generation tools.\n\nThere are many different types of LLMs, each with its own strengths and weaknesses. Some popular LLMs include:\n\n* BERT (Bidirectional Encoder Representations from Transformers): A widely used LLM developed by Google that is known

In [67]:
agent_exe.invoke({"input":"what is langchain"})



> Entering new AgentExecutor chain...

Invoking: `langchin-search` with `{'query': 'langchain'}`


Providers | 🦜️🔗 LangChain

Providers | 🦜️🔗 LangChain

infoIf you'd like to write your own integration, see Extending LangChain.
If you'd like to contribute an integration, see Contributing integrations.
LangChain integrates with many providers.
Integration Packages​
These providers have standalone langchain-{provider} packages for improved versioning, dependency management and testing.

infoIf you'd like to write your own integration, see Extending LangChain.
If you'd like to contribute an integration, see Contributing integrations.
LangChain integrates with many providers.
Integration Packages​
These providers have standalone langchain-{provider} packages for improved versioning, dependency management and testing.
Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language model

{'input': 'what is langchain',
 'output': 'This suggests that LangChain is a software framework that helps integrate large language models (LLMs) into applications, including document analysis, text generation, and more. It also appears to have integrations with various providers, including Wikipedia and Arxiv. Additionally, there are concerns about the security of LLM-integrated web applications, which LangChain may be able to address.'}

In [68]:
agent_exe.invoke({"input":"what is attention all you need paper is all about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention All You Need paper is all about'}`


Published: 2018-07-02
Title: The Matrix Calculus You Need For Deep Learning
Authors: Terence Parr, Jeremy Howard
Summary: This paper is an attempt to explain all the matrix calculus you need in order
to understand the training of deep neural networks. We assume no math knowledge
beyond what you lear
Invoking: `wikipedia` with `{'query': 'Attention All You Need paper'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014It appears that the paper "Attention All You Need" you are referring to is actually called "Attention Is All You Need". This paper introduced the Transformer model, which is a type of neural network arch

{'input': 'what is attention all you need paper is all about?',
 'output': 'It appears that the paper "Attention All You Need" you are referring to is actually called "Attention Is All You Need". This paper introduced the Transformer model, which is a type of neural network architecture that does not rely on recurrent neural networks (RNNs) or convolutional neural networks (CNNs) for sequential data. Instead, it uses self-attention mechanisms to process input sequences in parallel, making it highly efficient and scalable for tasks such as machine translation, text classification, and question answering.'}